In [1]:
# hidden num remain unchange, Using Surgery
import torch
old = torch.load('/home/workspace/util/surgery/titanic_demo/runs/May24_08-22-52_1d7a14f7cd78old_network with old input from scratch/oldnetwork_oldinput_from_scratch_bs32.ckpt')

In [11]:
import os
import pandas as pd
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
from torch import optim
from pretreat import *
%matplotlib inline
np.random.seed(0)
training = get_pretreated_training_data()
old_input_dict = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'cabine_n', 'IsAlone']
loader = get_dataloader(training, old_input_dict, bs=32)
model = MLP(input_dim = len(old_input_dict), hidden_dims=[40, 50, 100, 100, 100, 100,]).cuda()

/home/miniconda/envs/tftorch/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [12]:
for i, (name, g) in enumerate(model.named_parameters()):
    if name == 'layers.0.weight':
        print(name, g[20:30, :2])
    if name == 'layers.0.bias':
        print(name, g[20:30])
    # print(name, g[:2]) if i%2 else print(name, g[:2, :2])

layers.0.weight tensor([[ 0.1539,  0.0256],
        [-0.3500,  0.2779],
        [-0.0202,  0.1378],
        [ 0.1163, -0.0836],
        [-0.2597, -0.2726],
        [-0.1182, -0.3295],
        [ 0.3205, -0.1218],
        [-0.1202,  0.0017],
        [-0.3518,  0.2374],
        [-0.1666, -0.1704]], device='cuda:0', grad_fn=<SliceBackward>)
layers.0.bias tensor([ 0.1186, -0.0353,  0.1577,  0.3102,  0.1784,  0.3436,  0.1144,  0.0822,
        -0.1515, -0.0894], device='cuda:0', grad_fn=<SliceBackward>)


In [13]:
# 只增加中间层, 那么只需要对多出来的进行0初始化就好了, 
for i, (name, g) in enumerate(model.named_parameters()):
    old_g = old['para'][name].data
    g.data = g.data.zero_()
    if i%2:
        d = old_g.shape
        g.data[:oi] = old_g
    else:
        oi, oo = old_g.shape
        g.data[:oi,:oo] = old_g

In [15]:
for i, (name, g) in enumerate(model.named_parameters()):
    if name == 'layers.0.weight':
        print(name, g[20:30, :2])
    if name == 'layers.0.bias':
        print(name, g[20:30])
    # print(name, g[:2]) if i%2 else print(name, g[:2, :2])

layers.0.weight tensor([[-4.5212e-07, -1.1475e-02],
        [-7.5477e-01,  2.4377e-02],
        [ 7.2729e-41, -2.0507e-10],
        [-8.0097e-01, -1.7311e-01],
        [-3.5086e-01, -1.5174e-01],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00]], device='cuda:0', grad_fn=<SliceBackward>)
layers.0.bias tensor([-3.4280e-13, -4.6570e-01, -3.5227e-41,  8.5131e-02,  2.4277e-01,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
       device='cuda:0', grad_fn=<SliceBackward>)


In [16]:
# get surgery之后最初始的acc
sum_acc = 0
for x, y in loader:
    x, y = x.cuda(), y.cuda()
    outputs=model(x)
    _,id=torch.max(outputs.data,1)
    acc=torch.sum(id==y.data)
    sum_acc += acc.item()    
print(sum_acc)

/home/miniconda/envs/tftorch/lib/python3.6/site-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


752


In [17]:
lr = 0.001
epoches = 2000 
comment = 'new_network with old input Using Surgery'
model_name = 'newnetwork_oldinput_usingSurgery.ckpt'
input_dic = old_input_dict
acc_cur, loss_cur = train(model, loader, comment, model_name, input_dic, )

/home/miniconda/envs/tftorch/lib/python3.6/site-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch:0, loss:13.409809857606888, acc:0.8338945005611672
